# First Section is the original task 5 - See below for Work Zone

# Quantum Circuits
Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.

In [1]:
import tequila as tq
from utility import *

First, we set up the Hamiltonian in Tequila's format and the unitary gates obtained in Step 3. 

In [2]:
H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian('h2', 2, 'sto-3g', qubit_transf='jw')) 

a = tq.Variable("tau_0")
U = construct_QMF_ansatz(4)
U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string("X(0)Y(1)X(2)X(3)"), angle=a)
print(U)

circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Exp-Pauli(target=(0, 1, 2, 3), control=(), parameter=tau_0, paulistring=X(0)Y(1)X(2)X(3))



One can check the expectation value to see it is near the ground state energy.

In [3]:
E = tq.ExpectationValue(H=H, U=U)
vars = {'beta_1': 3.141592624143881, 'beta_0': 3.141592624143881, 'tau_0': 1.1331410014096885, 'gamma_1': 0.0, 'beta_3': 0.0, 'gamma_3': 0.0, 'gamma_2': 0.0, 'gamma_0': 0.0, 'beta_2': 0.0} # values obtained from step 3
print(tq.simulate(E, variables=vars))

-0.9486411121761622


One can run the same experiment on a real quantum computer through IBM Quantum Experience (ibmq). After activating your account here (https://quantum-computing.ibm.com/login), copy the API token and execute the commented block below. 

In [ ]:
from qiskit import IBMQ
IBMQ.save_account('')

In [4]:
# list of devices available can be found in ibmq account page
tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_qasm_simulator')

-0.9526530610853907

The following code block prints the circuit.

In [5]:
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")

     ┌────────────────────┐┌─────────────────────┐   ┌───┐                   »
q_0: ┤ RX(f((beta_0,))_0) ├┤ RZ(f((gamma_0,))_1) ├───┤ H ├──────■────────────»
     ├────────────────────┤├─────────────────────┤┌──┴───┴───┐┌─┴─┐          »
q_1: ┤ RX(f((beta_1,))_2) ├┤ RZ(f((gamma_1,))_3) ├┤ RX(pi/2) ├┤ X ├──■───────»
     ├────────────────────┤├─────────────────────┤└──┬───┬───┘└───┘┌─┴─┐     »
q_2: ┤ RX(f((beta_2,))_4) ├┤ RZ(f((gamma_2,))_5) ├───┤ H ├─────────┤ X ├──■──»
     ├────────────────────┤├─────────────────────┤   ├───┤         └───┘┌─┴─┐»
q_3: ┤ RX(f((beta_3,))_6) ├┤ RZ(f((gamma_3,))_7) ├───┤ H ├──────────────┤ X ├»
     └────────────────────┘└─────────────────────┘   └───┘              └───┘»
c_0: ════════════════════════════════════════════════════════════════════════»
                                                                             »
c_1: ════════════════════════════════════════════════════════════════════════»
                                                    

# Work zone - Project 2 - Task 5


In [6]:
import tequila as tq
from utility import *

#
# Molecules to process
#

class p2t5:
    def __init__(self):
        self.molecules = [] 
        self.molecules.append("h2".upper())
        self.molecules.append("h2o".upper())
        self.molecules.append("lih".upper())
        self.molecules.append("h4".upper())
        self.molecules.append("n2".upper())
        self.molecules.append("nh3".upper())
        print(self.molecules)
        
    def haveMol(self,name):

        name = name.upper()
        for m in range(len(self.molecules)):
            if ( self.molecules[m] == name ):
                return True
        return False

    # Experienting zone to abstract parameters for various molecules 

    def calculate_QCC_energy(self,molname,geometry=2.5,basis='sto-3g',n_ents=1,method="BFGS", active_orbitals=None, threshold=1e-6):
            if ( self.haveMol(molname)):

                print("\n\nCalculating QCC for ", molname.upper())
                print("Basis: {}\nGeometry: {}\nEntanglers: {}\nActive Orbitals: {}".format(basis,geometry,n_ents,active_orbitals))
                
                # Testing
                
                xyz_data = get_molecular_data(molname, geometry=geometry, xyz_format=True)
                mol = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals = active_orbitals)
                hf_reference = hf_occ(2*mol.n_orbitals, mol.n_electrons)
                
                # FCI energy calculation
                
                E_FCI = mol.compute_energy(method='fci')
                
                # Hamiltonian
                
                H = mol.make_hamiltonian()

                print("\nHamiltonian has {} terms\n".format(len(H)))

                #Define number of entanglers to enter ansatz
                

                #Rank entanglers using energy gradient criterion
                ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                                             2*mol.n_orbitals, 
                                                                             hf_reference, 
                                                                             cutoff=threshold)

                print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
                for i in range(len(ranked_entangler_groupings)):
                    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


                entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*mol.n_orbitals)

                print('\nSelected entanglers:')
                for ent in entanglers:
                    print(ent)
                    
                # 
                #Mean-field part of U (Omega):    
                U_MF = construct_QMF_ansatz(n_qubits = 2*mol.n_orbitals)
                
                #Entangling part of U:
                U_ENT = construct_QCC_ansatz(entanglers)

                U_QCC = U_MF + U_ENT

                E = tq.ExpectationValue(H=H, U=U_QCC)

                initial_vals = init_qcc_params(hf_reference, E.extract_variables())
                print("Initial Values: {}".format(initial_vals))

                #Minimize wrt the entangler amplitude and MF angles:
                result = tq.minimize(objective=E, method=method, initial_values=initial_vals, tol=1.e-6)

                print('\nObtained QCC energy for {} ({} entanglers): {}'.format(molname.upper(),len(entanglers), result.energy))
                print('Compare to FCI energy: {}\n'.format(E_FCI))
                
                # Memorize the last compute
                
                self.comp_n_ents = n_ents
                self.comp_basis = basis
                self.comp_mol = molname
                self.comp_E_QCC = result.energy
                self.comp_entanglers = entanglers
                self.comp_E_FCI = E_FCI
                self.comp_initial_vals = initial_vals
                
            else:
                print("molecule not known")

    def calculateCircuit(self):
        # Utilize results from calculate_QCC_energy
        
        print("Constructing circuit(s) for {}".format(self.comp_mol.upper()))
        
        H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian(self.comp_mol, 2, self.comp_basis, qubit_transf='jw')) 
        a = tq.Variable("tau_0") # Why do we pick tau ??? This is for H2. What about others?
        
        Ux = []
        for e in self.comp_entanglers:

            U = construct_QMF_ansatz(4) # Why is 4 selected?

            # For each selected entanglers we will need to compute a separate circuit
            # We we need to create an array of gates
            # we will need to transform the result from Step 3 into a result that can be used here
            #  [X0 Y1 X2 X3] ==> X(0)Y(1)X(2)X(3)
        
            # Base circuit
            #U = base_U
            toString = ""
            for t in e.terms:
                for c,axis in t:
                    toString += axis + "(" + str(c) + ")"
            # print (toString)
            U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string(toString), angle=a)
            Ux.append(U)
        
        print("============= {} Circuits to execute =============".format(len(Ux)))
        
        c_i = 0
        for circuit in Ux:
            print("Circuit #", c_i)
            c_i = c_i + 1
              
            print(circuit)
            E = tq.ExpectationValue(H=H, U=circuit)
            vars =self.comp_initial_vals
            print(tq.simulate(E, variables=vars))
            
            #print("Calculated vars here      : ", vars)
            
            
            
            

In [7]:
obj = p2t5()

obj.calculate_QCC_energy("h2", geometry=2.5, n_ents = 1)
obj.calculateCircuit()

obj.calculate_QCC_energy("h2o", geometry=1,  n_ents = 6, basis="6-31g", active_orbitals = {'B1':[0,1], 'A1':[2,3]})
obj.calculateCircuit()

#obj.calculate_QCC_energy("lih")
#obj.calculate_QCC_energy("h4")
#obj.calculate_QCC_energy("n2")
#obj.calculate_QCC_energy("nh3")

['H2', 'H2O', 'LIH', 'H4', 'N2', 'NH3']


Calculating QCC for  H2
Basis: sto-3g
Geometry: 2.5
Entanglers: 1
Active Orbitals: None

Hamiltonian has 15 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.2822

Selected entanglers:
1.0 [X0 Y1 X2 X3]
Initial Values: {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.0}
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 18 expectationvalues

active variables : 9

E=-0.70294360  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.0}  samples= None
E=-0.77938639  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.1415926535897